In this empiric test we look at 'runs' in the random sequence. A run is a strict strictly monotonically increasing subsequence. 

    sequence                 sub sequences            runs           run counts
    1,2,9,8,5,3,6,7,0,4  ->  1,2,9|8|5|3,6,7|0,4  ->  3,1,1,3,2  ->  2,1,2,0,0,0
    
Since adjacent run counts are not independent, we can not directly apply a chi-square test, but must use a special statistic

    V = 1/(n-6) \sum_{i=1}^{6}\sum_{j=1}^{6} (count[i] - n*B[i]) (count[j] - n*B[j]) a[i][j]

With special matrices A and B, that he explains how to derrive.

The statistic V should have a chi-square distribution with 6 degrees of freedom.
    
    0.90 -> 10.645
    0.95 -> 12.59

## Code

In [115]:
import itertools
from collections import Counter
import random

def getPseudoRandomSequence(x0, k, m):
    last = x0
    while True:
        last = k * last % m
        yield last/m
def getFinitePseudorandomSequence(x0, k, m, n):
    return itertools.islice(getPseudoRandomSequence(x0, k, m),n)

def getPythonRandom():
    while True:
        yield random.random()
    
def getOsRandom():
    os_random = random.SystemRandom()
    while True:
        yield os_random.random()

In [3]:
def getRuns(sequence, condition):
    currentRun = []
    for element in sequence:
        if not currentRun or condition(currentRun[-1], element):
            currentRun.append(element)
        else:
            yield currentRun
            currentRun = [element]
    yield currentRun
def getIncreasingRuns(sequence):
    return getRuns(sequence, lambda a,b: a<b)
def getDecreasingRuns(sequence):
    return getRuns(sequence, lambda a,b: a>b)

def getRunLenghts(runs):
    for element in runs:
        yield len(element)
def getIncreasingRunLenghts(sequence):
    return getRunLenghts(getIncreasingRuns(sequence))
def getDecreasingRunLenghts(sequence):
    return getRunLenghts(getDecreasingRuns(sequence))

def getRunLenghtCounts(runLenghts):
    counter = dict(Counter(runLenghts))
    counts = [counter[key] for key in sorted(counter.keys())]
    counts[5:] = [sum(counts[5:])]
    return counts
def getIncreasingRunLenghtCounts(sequence):
    return getRunLenghtCounts(getRunLenghts(getIncreasingRuns(sequence)))
def getDecreasingRunLenghtCounts(sequence):
    return getRunLenghtCounts(getRunLenghts(getDecreasingRuns(sequence)))

In [4]:
A = [[  4529.4,  9044.9, 13568.0,  18091.0,  22615.0,  27892.0], 
     [  9044.9, 18097.0, 27139.0,  36187.0,  45234.0,  55789.0],
     [ 13568.0, 27139.0, 40721.0,  54281.0,  67852.0,  83685.0],
     [ 18091.0, 36187.0, 54281.0,  72414.0,  90470.0, 111580.0],
     [ 22615.0, 45234.0, 67852.0,  90470.0, 113262.0, 139476.0],
     [ 27892.0, 55789.0, 83685.0, 111580.0, 139476.0, 172860.0]]
B = [1/6, 5/24, 11/120, 19/720, 29/5040, 1/840]

In [5]:
def getStatisticV(count, n):
    return 1/(n-6) * sum([(count[i]-n*B[i])*(count[j]-n*B[j])*A[i][j] for i in range(0,6) for j in range(0,6)])

In [6]:
# sequence = [1,2,9,8,5,3,6,7,0,4]
# sequence = [1,3,8,7,5,2,6,7,1,6]
x0 = 1001
k = 8192
m = 67101323
n = 10000
sequence = list(getFinitePseudorandomSequence(x0, k, m, n))
print(getStatisticV(getIncreasingRunLenghtCounts(sequence), n))
print(getStatisticV(getDecreasingRunLenghtCounts(sequence), n))

11.75899989477215
7.51414663360855


## Test Runs

Values for generated sequences taken from:
```
The Runs-Up and Runs-Down Tests By R. G. T. GRAFTON
Journal of the Royal Statistical Society. Series C (Applied Statistics), Vol. 30, No. 1 (1981), pp. 81-85```

In [116]:
s1 = getPseudoRandomSequence(x0=1001, k=8192, m= 67101323)
s2 = getPseudoRandomSequence(x0=1001, k=8192, m= 67099547)
s3 = getPseudoRandomSequence(x0=1001, k=32768, m= 16775723)
s4 = getPseudoRandomSequence(x0=1001, k=54751, m= 99707)
s5 = getPseudoRandomSequence(x0=1001, k=8, m= 67100963)
s6 = getPseudoRandomSequence(x0=1001, k=32, m= 7999787)
s7 = getPythonRandom()
s8 = getOsRandom()
size = [4000,10000,15000,20000,25000,30000,35000,40000]
size.extend(size)
size.extend(size)
size.sort()

In [117]:
import plotly

def getCoordsForGraph(sequence, size):
#     sequence = list(itertools.islice(sequence, max(size)))
#     data = {
#         'up': [getStatisticV(getIncreasingRunLenghtCounts(sequence[:n]), n) for n in size],
#         'down': [getStatisticV(getDecreasingRunLenghtCounts(sequence[:n]), n) for n in size],
#     }
#     print(data)
    graphs = {}
    graphs['sequence'] = plotly.graph_objs.Scatter(
        x = list(range(1, max(size)+1)), 
        y = list(itertools.islice(sequence, max(size))),
        mode = 'markers',
        marker = dict(size=1, line=dict(width=0), color='blue'),
        name = 'sequence'
    )
    graphs['up'] = plotly.graph_objs.Scatter(
        x = size, 
        y = [getStatisticV(getIncreasingRunLenghtCounts(itertools.islice(sequence, n)), n) for n in size],
        mode = 'markers',
        marker = dict(size=8, line=dict(width=1), color='red'),
        name = 'runs up'
    )
    graphs['down'] = plotly.graph_objs.Scatter(
        x = size, 
        y = [getStatisticV(getDecreasingRunLenghtCounts(itertools.islice(sequence, n)), n) for n in size],
        mode = 'markers',
        marker = dict(size=8, line=dict(width=1), color='blue'),
        name = 'runs down'
    )
    return graphs

def plotRunTest(sequence, size, name, plot_sequence=False): 
    plot = getCoordsForGraph(sequence, size)
    layout = plotly.graph_objs.Layout(
        title=name,
        xaxis = dict(
            title='Size of sequence'
        ),
        yaxis = dict(
            title='Statistic V'
        ),
        shapes = [
            { # horizontal line for .90
                'type': 'line',
                'x0': 0,
                'y0': 10.645,
                'x1': 45000,
                'y1': 10.645,
                'line': {
                    'color': 'orange',
                    'width': 3,
                    'dash': 'longdash'
                }
            },
            { # horizontal line for .95
                'type': 'line',
                'x0': 0,
                'y0': 12.59,
                'x1': 45000,
                'y1': 12.59,
                'line': {
                    'color': 'darkred',
                    'width': 3,
                    'dash': 'dash'
                }
            }
        ]
    )
    
    if plot_sequence:
        plotly.plotly.plot(plotly.graph_objs.Figure(
                data = [plot['sequence']],
                layout = dict(
                    title = name
                )
            ),
            filename = 'sequence',
            auto_open = True
        )
                           
    plotly.plotly.plot(plotly.graph_objs.Figure(
            data = [plot['up'], plot['down']], 
            layout = layout
        ),
        filename = name, 
        auto_open = True
    )

In [99]:
plotRunTest(s1, size, 'Sequence1', True)

The draw time for this plot will be slow for clients without much RAM.


In [100]:
plotRunTest(s2, size, 'Sequence2', True)

The draw time for this plot will be slow for clients without much RAM.


In [101]:
plotRunTest(s3, size, 'Sequence3', True)

The draw time for this plot will be slow for clients without much RAM.


In [102]:
plotRunTest(s4, size, 'Sequence4', True)

The draw time for this plot will be slow for clients without much RAM.


In [103]:
plotRunTest(s5, size, 'Sequence5', True)

The draw time for this plot will be slow for clients without much RAM.


In [118]:
plotRunTest(s6, size, 'Sequence6', True)

The draw time for this plot will be slow for clients without much RAM.


In [119]:
plotRunTest(s7, size, 'Python Random', True)

The draw time for this plot will be slow for clients without much RAM.


In [120]:
plotRunTest(s8, size, 'OS Random', True)

The draw time for this plot will be slow for clients without much RAM.
